In [7]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: c:\Users\Admin\.pyenv\pyenv-win\versions\3.11.9\python.exe -m pip install --upgrade pip


In [9]:
pip install PyMySQL[rsa]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: c:\Users\Admin\.pyenv\pyenv-win\versions\3.11.9\python.exe -m pip install --upgrade pip


In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import time
import pymysql
from datetime import datetime
import random  # 더미 데이터 생성을 위해 추가

# ===============================
# 1. MySQL DB 연결 정보
# ===============================
DB_HOST = "localhost"
DB_PORT = 3306
DB_USER = "root"
DB_PASS = "12345"
DB_NAME = "electric_db"
TABLE_HISTORY = "전력량"
TABLE_REALTIME = "시간"

def get_connection():
    return pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASS,
        database=DB_NAME,
        charset="utf8",
        client_flag=pymysql.constants.CLIENT.MULTI_STATEMENTS,  # 다중 쿼리 허용
        ssl_disabled=True,  # SSL 비활성화 (caching_sha2_password 방지)
        auth_plugin_map={'caching_sha2_password':'mysql_native_password'}
    )

# ===============================
# 2. 과거 데이터 불러오기 (경계값 계산)
# ===============================
conn = get_connection()
df = pd.read_sql_query(
    "SELECT 시간, 전력량 FROM 전력량 ORDER BY 시간 ASC;", 
    conn
)
conn.close()

df.columns = ['date', 'price']
df["price"] = pd.to_numeric(df["price"], errors="coerce")  # 숫자 변환
df = df.dropna(subset=["price"])  # NaN 제거

if df["price"].empty:
    raise ValueError("오류: 'price' 열에 데이터가 없습니다.")

scaler = MinMaxScaler()
scaler.fit(df["price"].values.reshape(-1, 1))

boundaries = {
    "상위 5%": np.percentile(df["price"], 95),
    "상위 10%": np.percentile(df["price"], 90),
    "상위 25%": np.percentile(df["price"], 75),
    "하위 5%": np.percentile(df["price"], 5),
    "하위 10%": np.percentile(df["price"], 10),
    "하위 25%": np.percentile(df["price"], 25)
}

def classify_price(price):
    if price >= boundaries["상위 5%"]:
        return "1등급"
    elif price >= boundaries["상위 10%"]:
        return "2등급"
    elif price >= boundaries["상위 25%"]:
        return "3등급"
    elif price <= boundaries["하위 5%"]:
        return "7등급"
    elif price <= boundaries["하위 10%"]:
        return "6등급"
    elif price <= boundaries["하위 25%"]:
        return "5등급"
    else:
        return "4등급"

# ===============================
# 3. LSTM 모델 로드
# ===============================
MODEL_PATH = "electricity_lstm_model.h5"

try:
    model = load_model(MODEL_PATH, compile=False)  # 최신 Keras 호환
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    print("✅ 모델 로드 성공!")
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("새 모델을 생성합니다...")
    model = Sequential([
        LSTM(50, activation='relu', input_shape=(10, 1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    print("✅ 새 모델 생성 완료")

# ===============================
# 4. 아두이노 대신 랜덤 데이터 생성
# ===============================
SEQ_LEN = 10
recent_data = []

print("실시간 전력 요금 예측 + MySQL 저장 시작... (Ctrl+C 종료)")

try:
    while True:
        # 더미 전력 사용량 (0.1~5.0 kWh 범위)
        usage_kwh = round(random.uniform(0.1, 5.0), 2)
        price = usage_kwh * 100  # 단가 100원/kWh
        recent_data.append(price)

        if len(recent_data) >= SEQ_LEN:
            seq_input = np.array(recent_data[-SEQ_LEN:]).reshape(-1, 1)
            seq_scaled = scaler.transform(seq_input)
            seq_scaled = np.expand_dims(seq_scaled, axis=0)

            pred_scaled = model.predict(seq_scaled, verbose=0)
            pred_price = scaler.inverse_transform(pred_scaled)[0][0]

            category = classify_price(pred_price)

            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"[{now}] 예측 요금: {pred_price:,.0f}원 → {category}")

            conn = get_connection()
            cur = conn.cursor()
            insert_sql = f"""
                INSERT INTO {TABLE_REALTIME} (timestamp, usage_kwh, predicted_price, category)
                VALUES (%s, %s, %s, %s)
            """
            cur.execute(insert_sql, (now, usage_kwh, pred_price, category))
            conn.commit()
            conn.close()

        time.sleep(1)

except KeyboardInterrupt:
    print("종료합니다.")


C:\Users\Admin\AppData\Local\Temp\ipykernel_6156\2388948537.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(


✅ 모델 로드 성공!
실시간 전력 요금 예측 + MySQL 저장 시작... (Ctrl+C 종료)
[2025-08-14 10:11:24] 예측 요금: 18,107원 → 1등급


ProgrammingError: (1146, "Table 'electric_db.시간' doesn't exist")

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import time
import pymysql
from datetime import datetime
import random

# ===============================
# 1. MySQL DB 연결 정보
# ===============================
DB_HOST = "localhost"
DB_PORT = 3306
DB_USER = "root"
DB_PASS = "12345"
DB_NAME = "electric_db"
TABLE_HISTORY = "전력량"
TABLE_REALTIME = "시간"  # 실시간 저장 테이블

def get_connection():
    return pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASS,
        database=DB_NAME,
        charset="utf8"
    )

def ensure_table_exists():
    """TABLE_REALTIME 테이블이 없으면 자동 생성"""
    conn = get_connection()
    cur = conn.cursor()
    create_sql = f"""
    CREATE TABLE IF NOT EXISTS `{TABLE_REALTIME}` (
        id INT AUTO_INCREMENT PRIMARY KEY,
        timestamp DATETIME,
        usage_kwh FLOAT,
        predicted_price FLOAT,
        category VARCHAR(10)
    ) CHARACTER SET utf8mb4;
    """
    cur.execute(create_sql)
    conn.commit()
    conn.close()

# ===============================
# 2. 과거 데이터 불러오기 (경계값 계산)
# ===============================
conn = get_connection()
# 과거 데이터 테이블이 없으면 더미 데이터 생성
try:
    df = pd.read_sql_query(
        f"SELECT 시간, 전력량 FROM {TABLE_HISTORY} ORDER BY 시간 ASC;",
        conn
    )
except pymysql.err.ProgrammingError:
    # 전력량 테이블 없으면 생성 + 더미 데이터 채우기
    cur = conn.cursor()
    cur.execute(f"""
    CREATE TABLE IF NOT EXISTS `{TABLE_HISTORY}` (
        시간 DATETIME,
        전력량 FLOAT
    ) CHARACTER SET utf8mb4;
    """)
    conn.commit()

    now = datetime.now()
    dummy_data = [
        (now.replace(minute=0, second=0, microsecond=0) - pd.Timedelta(hours=i), round(random.uniform(0.5, 3.0), 2))
        for i in range(100)
    ]
    cur.executemany(f"INSERT INTO `{TABLE_HISTORY}` (시간, 전력량) VALUES (%s, %s)", dummy_data)
    conn.commit()

    df = pd.DataFrame(dummy_data, columns=["date", "price"])
else:
    df.columns = ['date', 'price']
conn.close()

df["price"] = pd.to_numeric(df["price"], errors="coerce")
df = df.dropna(subset=["price"])

if df["price"].empty:
    raise ValueError("오류: 'price' 열에 데이터가 없습니다.")

scaler = MinMaxScaler()
scaler.fit(df["price"].values.reshape(-1, 1))

# boundaries = {
#     "상위 5%": np.percentile(df["price"], 95),
#     "상위 10%": np.percentile(df["price"], 90),
#     "상위 25%": np.percentile(df["price"], 75),
#     "하위 5%": np.percentile(df["price"], 5),
#     "하위 10%": np.percentile(df["price"], 10),
#     "하위 25%": np.percentile(df["price"], 25)
# }

boundaries = {
    "상위 5%": 50000,
    "상위 10%": 40000,
    "상위 25%": 30000,
    "하위 5%": 5000,
    "하위 10%": 10000,
    "하위 25%": 20000
}


# def classify_price(price):
#     if price >= boundaries["상위 5%"]:
#         return "1등급"
#     elif price >= boundaries["상위 10%"]:
#         return "2등급"
#     elif price >= boundaries["상위 25%"]:
#         return "3등급"
#     elif price <= boundaries["하위 5%"]:
#         return "7등급"
#     elif price <= boundaries["하위 10%"]:
#         return "6등급"
#     elif price <= boundaries["하위 25%"]:
#         return "5등급"
#     else:
#         return "4등급"
    
def classify_price(price):
    if price >= boundaries["상위 5%"]:
        return "1등급"
    elif price >= boundaries["상위 10%"]:
        return "2등급"
    elif price >= boundaries["상위 25%"]:
        return "3등급"
    elif price <= boundaries["하위 5%"]:
        return "7등급"
    elif price <= boundaries["하위 10%"]:
        return "6등급"
    elif price <= boundaries["하위 25%"]:
        return "5등급"
    else:
        return "4등급"

# ===============================
# 3. LSTM 모델 로드
# ===============================
MODEL_PATH = "electricity_lstm_model.h5"

try:
    model = load_model(MODEL_PATH, compile=False)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    print("✅ 모델 로드 성공!")
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("새 모델을 생성합니다...")
    model = Sequential([
        LSTM(50, activation='relu', input_shape=(10, 1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    print("✅ 새 모델 생성 완료")

# ===============================
# 4. 실시간 예측 시작 (더미 데이터)
# ===============================
ensure_table_exists()
SEQ_LEN = 10
recent_data = []

print("실시간 전력 요금 예측 + MySQL 저장 시작... (Ctrl+C 종료)")

try:
    while True:
        usage_kwh = round(random.uniform(0.1, 5.0), 2)
        price = usage_kwh * 100
        recent_data.append(price)

        if len(recent_data) >= SEQ_LEN:
            seq_input = np.array(recent_data[-SEQ_LEN:]).reshape(-1, 1)
            seq_scaled = scaler.transform(seq_input)
            seq_scaled = np.expand_dims(seq_scaled, axis=0)

            pred_scaled = model.predict(seq_scaled, verbose=0)
            pred_price = scaler.inverse_transform(pred_scaled)[0][0]

            category = classify_price(pred_price)

            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"[{now}] 예측 요금: {pred_price:,.0f}원 → {category}")

            conn = get_connection()
            cur = conn.cursor()
            insert_sql = f"""
                INSERT INTO `{TABLE_REALTIME}` (timestamp, usage_kwh, predicted_price, category)
                VALUES (%s, %s, %s, %s)
            """
            cur.execute(insert_sql, (now, usage_kwh, pred_price, category))
            conn.commit()
            conn.close()

        time.sleep(1)

except KeyboardInterrupt:
    print("종료합니다.")


C:\Users\Admin\AppData\Local\Temp\ipykernel_7828\2572121364.py:56: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(


✅ 모델 로드 성공!
실시간 전력 요금 예측 + MySQL 저장 시작... (Ctrl+C 종료)
[2025-08-21 14:21:15] 예측 요금: 33,622원 → 3등급
[2025-08-21 14:21:16] 예측 요금: 37,322원 → 3등급
[2025-08-21 14:21:17] 예측 요금: 32,268원 → 3등급
[2025-08-21 14:21:18] 예측 요금: 39,488원 → 3등급
[2025-08-21 14:21:19] 예측 요금: 24,075원 → 4등급
[2025-08-21 14:21:20] 예측 요금: 25,920원 → 4등급
[2025-08-21 14:21:21] 예측 요금: 22,273원 → 4등급
[2025-08-21 14:21:22] 예측 요금: 27,998원 → 4등급
[2025-08-21 14:21:23] 예측 요금: 29,894원 → 4등급
[2025-08-21 14:21:24] 예측 요금: 37,550원 → 3등급
[2025-08-21 14:21:25] 예측 요금: 24,869원 → 4등급
[2025-08-21 14:21:26] 예측 요금: 35,147원 → 3등급
[2025-08-21 14:21:27] 예측 요금: 18,096원 → 5등급
[2025-08-21 14:21:29] 예측 요금: 25,767원 → 4등급
[2025-08-21 14:21:30] 예측 요금: 30,987원 → 3등급
[2025-08-21 14:21:31] 예측 요금: 23,288원 → 4등급
[2025-08-21 14:21:32] 예측 요금: 28,619원 → 4등급
[2025-08-21 14:21:33] 예측 요금: 19,078원 → 5등급
[2025-08-21 14:21:34] 예측 요금: 31,561원 → 3등급
[2025-08-21 14:21:35] 예측 요금: 42,647원 → 2등급
[2025-08-21 14:21:36] 예측 요금: 30,106원 → 3등급
[2025-08-21 14:21:37] 예측 요금: 21,909원 → 4등급


In [14]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 50)             │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)